# Mako Trading Home Test

In [7]:
# load required libraries
import sqlite3
import pandas as pd
import numpy as np

# Create SQL db connection
conn = sqlite3.connect(r'mako.db')

# set variables
db_table = 'daytrade'
csv_to_load = 'input.csv'
date= "14/10/2022"

In [8]:
# load CSV
df_main = pd.read_csv(csv_to_load, names=["timestamp","symbol","volume","price"])
# Convert to specific types
df_main[["volume", "price","timestamp"]] = df_main[["volume", "price","timestamp"]].apply(pd.to_numeric)
df_main[["symbol"]] = df_main[["symbol"]].astype(str)
#
df_main.head() 

,timestamp,symbol,volume,price
0,51300041417,fed,193,106
1,51300059250,bac,87,100
2,51300080561,aae,176,282
3,51300084194,ade,214,48
4,51300172329,gda,79,179


In [9]:
# sort, find differences per group and create new column
df = df_main[["timestamp","symbol"]].sort_values('timestamp', ascending=True)
df_main['timegap'] = df.groupby('symbol').diff()

# create a function to find weigthed averages
weighted_fn = lambda x: np.average(x, weights=df_main.loc[x.index, "volume"])
# Create aggregates
df_result = df_main.groupby(["symbol"]).agg(max_timegap=("timegap",'max'),
                                total_volume=("volume", "sum"),  
                                weigthed_average_price=("price", weighted_fn), 
                                max_price=("price","max"),
                               )
df_result[["weigthed_average_price", "max_timegap"]] = df_result[
    ["weigthed_average_price", "max_timegap"]].astype(int)
#
df_result.head()

,max_timegap,total_volume,weigthed_average_price,max_price
symbol,,,,
aaa,9842035,8024,5,12
aab,9863672,9939,192,290
aac,9910381,9287,6,19
aad,9945552,9806,205,353
aae,9995907,10325,455,735


In [10]:
# create table and add data
df_result.to_sql(db_table, conn, if_exists='replace')

343

In [11]:
# Fetch 10 biggest symbols by total volume of this day
query = '''SELECT * FROM {db_table}
ORDER BY total_volume DESC 
LIMIT 10
'''.format(db_table=db_table)

pd.read_sql_query(query, conn, index_col='symbol')

,max_timegap,total_volume,weigthed_average_price,max_price
symbol,,,,
ecg,9475787,12425,118,247
gga,9838769,10880,46,111
dba,9845468,10862,236,394
caa,9993421,10845,317,456
fda,9377145,10805,197,261
bcg,9947643,10701,5,10
gad,9992908,10698,234,374
cbf,9798729,10692,83,144
gfe,9803402,10669,7,27


In [12]:
# in case we want to add a date and group by date
df_result_date = df_result
db_table_date = db_table + "date"
df_result_date['date'] = pd.to_datetime("14/10/2022", dayfirst=True)
df_result_date.to_sql(db_table_date, conn, if_exists='replace')

# Fetch 10 biggest symbols by total volume in case we had a date
query = '''
select * from (
    select symbol,
            max_timegap,
            total_volume,
            weigthed_average_price,
            max_price,
            date,
            row_number() over (partition by date order by total_volume desc) as rank
    from {db_table}) ranks
where rank <= 10
'''.format(db_table=db_table_date)

pd.read_sql_query(query, conn, index_col='date')    

,symbol,max_timegap,total_volume,weigthed_average_price,max_price,rank
date,,,,,,
2022-10-14 00:00:00,ecg,9475787,12425,118,247,1
2022-10-14 00:00:00,gga,9838769,10880,46,111,2
2022-10-14 00:00:00,dba,9845468,10862,236,394,3
2022-10-14 00:00:00,caa,9993421,10845,317,456,4
2022-10-14 00:00:00,fda,9377145,10805,197,261,5
2022-10-14 00:00:00,bcg,9947643,10701,5,10,6
2022-10-14 00:00:00,gad,9992908,10698,234,374,7
2022-10-14 00:00:00,cbf,9798729,10692,83,144,8
2022-10-14 00:00:00,gfe,9803402,10669,7,27,9
